In [ ]:
import   os
import   sys
sys.path.insert(0, os.path.abspath('.'))
from     ismrmrdUtils   import   rawMRutils
import   numpy          as       np
from     matplotlib     import   pyplot as plt
import   math

In [ ]:
def regularizeSensitivityMap (scaledArrayData):

   '''
      This routine should take fully sampled array coil data, already
      scaled with either its own square root of sum of squares of the
      individual coil images, or body coil data.

      The approach below computes a cosine window over the supported
      k-space region, converts the image data back into k-space data,
      applies the cosine weighting, converts back into image space,
      and is then returned.
   '''

   # First, compute cosine kernel over k-space
   smoothingKernelData = np.zeros(scaledArrayData.shape, dtype=np.float64)
   # smoothingKernelData.shape

   resX = smoothingKernelData.shape[0]
   resY = smoothingKernelData.shape[1]

   midX = resX/2
   midY = resY/2

   maxRadius = min (midX, midY) * 0.5

   for y in range(0, resY):
      for x in range (0, resX):
         offX = np.double(x) - np.double (midX)
         offY = np.double(y) - np.double (midY)

         radius = np.sqrt ((offX * offX) + (offY * offY))

         if (radius < maxRadius):
            # Hann window
            # smoothingKernelData[x, y] = 0.5 - (0.5 * np.cos(np.pi * 2.0 * radius/maxRadius))
            # Simple cosine window
            smoothingKernelData[x, y] = 0.5 + 0.5 * np.cos(np.pi * 1.0 * radius/maxRadius)
         else:
            smoothingKernelData[x, y] = 0.0

   # computeAndPlot(smoothingKernelData)

   # Convert image space sensitivity map to k-space, and apply the above
   # computed (here - cosine) window
   kspaceData = np.fft.fftshift(np.fft.ifft2(np.fft.fftshift(scaledArrayData, axes=(0,1)), axes=(0,1)), axes=(0,1))

   windowedKSpace = np.multiply (smoothingKernelData, kspaceData)

   # fft back to image space and return smoothed map
   return np.fft.fftshift(np.fft.fft2(np.fft.fftshift(windowedKSpace, axes=(0,1)), axes=(0,1)), axes=(0,1))


## First - read in (fully sampled) calibration data from array coil

In [ ]:
# Put data reads from disk in their own cells as these don't need to be repeated
calDataArrayHeader, calDataArray = rawMRutils.returnHeaderAndData('./ScanArchive_20190529_090502374_converted.h5')

In [ ]:
calDataArrayImageSpace = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(calDataArray, axes=(1,2)), axes=(1,2)), axes=(1,2))
# rawMRutils.computeAndPlot(calDataArrayImageSpace, quant='phase', coil=6)

## Now - read in data from body coil.

In [ ]:
calDataBodyHeader, calDataBody = rawMRutils.returnHeaderAndData('./ScanArchive_20190529_090622648_converted.h5')

In [ ]:
# imageSpace = np.fft.fftshift(np.fft.fft2(calDataBody, axes=(1,2)), axes=(1,2))
calDataBodyImageSpace = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(calDataBody, axes=(1,2)), axes=(1,2)), axes=(1,2))
# rawMRutils.computeAndPlot(calDataBodyImageSpace, quant='phase')

## Read in noise data from array coil.

In [ ]:
noiseDataArrayHeader, noiseDataArray = rawMRutils.returnHeaderAndData('./ScanArchive_20190529_090902358_converted.h5')

In [ ]:
# noise data is from array coil, so *is* multi-coil by its very nature.  The coil dimension should be the very first
# dimension in the data array returned by 'rawMRutils.returnHeaderAndData'.
nCoils = np.shape(noiseDataArray)[0]
# Want to keep the coil dimension, but collapse across all others (hence, the '-1' below) to compute noise covariance
# matrix (i.e. psi, in Pruessmann paper - https://doi.org/10.1002/(SICI)1522-2594(199911)42:5<952::AID-MRM16>3.0.CO;2-S)
# between coils.
reshapedNoiseData = np.reshape(noiseDataArray, [nCoils, -1])
# np.shape(reshapedNoiseData)
noiseCov = np.cov(reshapedNoiseData)
# plt.imshow (np.sqrt(np.abs(noiseCov)))

### For unfolding matrix computation, need the inverse of the covariance matrix, i.e. $\psi$<sup>-1</sup>

In [ ]:
psiInv = np.linalg.inv(noiseCov)
plt.imshow (np.sqrt(np.abs(psiInv)))

### Get / set some useful data dimensions

In [ ]:
imageRows    = int(np.ceil(np.sqrt(nCoils)))
imageCols    = imageRows

dimRead      = np.shape(calDataArrayImageSpace)[1]
dimPhase     = np.shape(calDataArrayImageSpace)[2]
dimSlice     = np.shape(calDataArrayImageSpace)[3]
R            = 3   # reduction / acceleration factor
accelStride  = int(dimPhase/R)

In [ ]:
plottedCoilsNormalizedWithBody = plt.figure(figsize=(18,18))

bodyCoilMag = np.zeros(calDataBodyImageSpace.shape[:-1], dtype=float)

for z in range(dimSlice):
   bodyCoilMag[0, :, :, z] = np.abs(regularizeSensitivityMap(calDataBodyImageSpace[0,:,:,z,0]))

bodyCoilMax      = np.max(bodyCoilMag)
sensitivityImage = np.zeros(calDataArray.shape[:-1], dtype=complex)

for c in range (nCoils):
   for z in range (dimSlice):
      subImages = plottedCoilsNormalizedWithBody.add_subplot(imageRows, imageCols, c + 1)
      sensitivityImage[c,:,:,z] = (regularizeSensitivityMap(calDataArrayImageSpace[c,:,:,z,0] / bodyCoilMag[0,:,:,z]))
      tmp = np.abs(sensitivityImage[c,:,:,z])

      # This flattens all of the coil data into a single dimension, then sorts it
      tmpSorted = np.sort(tmp.flatten())

      sizeTmp = tmpSorted.size

      # subImages.imshow(tmp)

      # Then we take the 10th and 90th percentiles for displaying.  Without these, the sensitivity
      # profiles could not be seen with the default scaling.
      minTmp = tmpSorted[int(sizeTmp * 0.10)]
      maxTmp = tmpSorted[int(sizeTmp * 0.90)]

      # Threshold sensitivity map with (arbitrarily here - 25% of maximum intensity from body coil image)
      tmp[np.where (bodyCoilMag[0,:,:,z] < (0.25 * bodyCoilMax))] = 0

      if (z == 0):
         subImages.imshow(np.clip(tmp, minTmp, maxTmp))

### Create an artificially sub-sampled (accelerated) set of data from the array coil data (R specifed above)

In [ ]:
subSampledKSpace = np.zeros((calDataArray.shape), dtype=np.complex64)
subSampledKSpace[:, :, ::R, :, :] = calDataArray[:, :, ::R, :, :]
aliasedImageSpace = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(subSampledKSpace, axes=(1,2)), axes=(1,2)), axes=(1,2))
# for i in range (nCoils):
#    rawMRutils.computeAndPlot(aliasedImageSpace, quant='mag', coil=i)
rawMRutils.computeAndPlot(aliasedImageSpace, quant='mag', coil=-1)

In [ ]:
# Template SENSE recon in Matlab available at: https://users.fmrib.ox.ac.uk/~mchiew/docs/SENSE_tutorial.html

unfoldedImage = np.zeros(calDataArray.shape, dtype=np.complex64)

for t in range (aliasedImageSpace.shape[-1]):
   for z in range (dimSlice):
      # loop over the top-half of the image
      for y in range (accelStride):
         # loop over the entire left-right extent
         for x in range (dimRead):
            # pick out the sub-problem sensitivities
            sVector = sensitivityImage[:, x, y:(y + (R*accelStride)):accelStride, z]
            # solve the sub-problem in the least-squares sense.  'unFoldedElement' should have R elements
            # - representing the number of pixels separated/unfolded from any given pixel in the aliased
            # image, to their 'correct' positions in the unaliased image.
            unFoldedElement = np.dot(np.linalg.pinv(sVector), (aliasedImageSpace[:, x, y, z, t]))

            # for r in range(R):
               # unfoldedImage [0, x, y + r * accelStride, z, t] = unFoldedElement[r]

               # Just as a contrast, to show the unfolding works
               # unfoldedImage [:, x, y + r * accelStride, z, t] = aliasedImageSpace[:, x, y + r * accelStride, z, t]

            # Instead of above loop, utilize Python's list expansion
            unfoldedImage [0, x, y:(y + (R * accelStride)):accelStride, z, t] = unFoldedElement
            # Just as a contrast, to show the unfolding works
            # unfoldedImage [:, x, y:(y + (R * accelStride)):accelStride, z, t] = aliasedImageSpace[:, x, y:(y + (R * accelStride)):accelStride, z, t]

In [ ]:
rawMRutils.computeAndPlot(unfoldedImage, quant='mag', coil=-1)

### Now, comupte unfolding matrix ($U$), a la Pruessmann (doi: https://doi.org/10.1002/(SICI)1522-2594(199911)42:5%3C952::AID-MRM16%3E3.0.CO;2-S), equation 2:

### $U$ = (S<sup>H</sup>$\psi$<sup>-1</sup>S)<sup>-1</sup>S<sup>H</sup>$\psi$<sup>-1</sup>

Matrix multiplication should happen from right-most terms, moving to left ...

In [ ]:
# Create space for sensitivity matrix - should match with the volume being unfolded, so ignore time (last) dimension
# - i.e. [:-1]
uMatrix = np.zeros(aliasedImageSpace.shape[:-1], dtype=np.complex64)

for z in range (dimSlice):
   # loop over the 'accelStride' portion of the image
   for y in range (accelStride):
      for x in range (dimRead):
         # Now, take sensitivities by stacking pixels that would be super-imposed, when going from full FOV, to
         # reduced FOV, when accelerating (i.e. skipping k-space lines, i.e. reducing FOV in image space)
         sVector = sensitivityImage[:, x, y:(y + (R*accelStride)):accelStride, z]
         # now, construst U-matrix, a la Pruessmann
         sHermitian = np.transpose(np.conjugate(sVector))
         unFoldedElement = np.dot(np.linalg.inv(np.dot(sHermitian, np.dot(psiInv, sVector))),(np.dot(sHermitian, psiInv)))

         # fill in unfolding matrix
         # for r in range(R):
            # uMatrix[:, x, y + r * accelStride, z] = unFoldedElement[r]

         # Instead of above loop, utilize Python's list expansion
         uMatrix[:, x, y:(y + (R * accelStride)):accelStride , z] = np.transpose(unFoldedElement)

In [ ]:
# Now, apply unfolding matrix to aliased data - and results should be unfolded images.

unfoldedImageSpace = np.zeros((calDataArray.shape), dtype=np.complex64)

for t in range (aliasedImageSpace.shape[-1]):
   for z in range (dimSlice):
      # loop over the top-half of the image
      for y in range (dimPhase):
         # loop over the entire left-right extent
         for x in range (dimRead):
            unfoldedImageSpace[:, x, y, z, t] = np.dot(uMatrix[:,x, y, z], (aliasedImageSpace[:, x, y, z, t]))

In [ ]:
rawMRutils.computeAndPlot(unfoldedImageSpace, quant='mag', coil=-1)